In [1]:
import pandas as pd

In [4]:
Data = pd.read_csv('valeursfoncieres-2022.txt', sep='|')
Data2019 = pd.read_csv('valeursfoncieres-2019.txt', sep='|')

C:\Users\enzom\PycharmProjects\AnalyseESG\venv\lib\site-packages\IPython\core\interactiveshell.py:3505: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\enzom\PycharmProjects\AnalyseESG\venv\lib\site-packages\IPython\core\interactiveshell.py:3505: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
Data

,Identifiant de document,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,1,2.0,Appartement,NaN,24.0,1.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,84.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,S,NaN,88.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,03/01/2022,Vente,...,NaN,1,2.0,Appartement,NaN,140.0,3.0,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,04/01/2022,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3803880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30/12/2022,Vente,...,NaN,1,4.0,Local industriel. commercial ou assimilé,NaN,327.0,0.0,NaN,NaN,NaN
3803881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,19/12/2022,Vente,...,NaN,2,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN
3803882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,19/12/2022,Vente,...,NaN,2,2.0,Appartement,NaN,40.0,3.0,NaN,NaN,NaN
3803883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,30/12/2022,Vente,...,NaN,1,3.0,Dépendance,NaN,0.0,0.0,NaN,NaN,NaN


In [6]:
columns_to_keep = ['Date mutation','Nature mutation','Valeur fonciere','Code postal','Commune','Code departement','Code commune','Nombre de lots','Code type local','Type local','Surface reelle bati','Nombre pieces principales','Surface terrain']
Data['Date mutation'] = pd.to_datetime(Data['Date mutation'])
Data['Code departement'] = Data['Code departement'].astype(str)
Data = Data[columns_to_keep]
Data = Data.dropna()
Data['Valeur fonciere'] = pd.to_numeric(Data['Valeur fonciere'].str.replace(',', '.'))
Data

,Date mutation,Nature mutation,Valeur fonciere,Code postal,Commune,Code departement,Code commune,Nombre de lots,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
5,2022-06-01,Vente,255000.0,1560.0,MANTENAY-MONTLIN,1,230,0,1.0,Maison,108.0,5.0,649.0
8,2022-03-01,Vente,525000.0,1390.0,SAINT-ANDRE-DE-CORCY,1,333,0,4.0,Local industriel. commercial ou assimilé,424.0,0.0,628.0
9,2022-03-01,Vente,525000.0,1390.0,SAINT-ANDRE-DE-CORCY,1,333,0,2.0,Appartement,126.0,4.0,628.0
13,2022-03-01,Vente,140000.0,1000.0,BOURG-EN-BRESSE,1,53,0,1.0,Maison,100.0,4.0,796.0
21,2022-10-01,Vente,580000.0,1500.0,AMBRONAY,1,7,0,3.0,Dépendance,0.0,0.0,496.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3803693,2022-12-28,Vente,12281000.0,75011.0,PARIS 11,75,111,0,2.0,Appartement,78.0,3.0,265.0
3803694,2022-12-28,Vente,12281000.0,75011.0,PARIS 11,75,111,0,2.0,Appartement,17.0,1.0,265.0
3803695,2022-12-28,Vente,12281000.0,75011.0,PARIS 11,75,111,0,2.0,Appartement,49.0,3.0,265.0
3803705,2022-12-30,Vente,3100000.0,75012.0,PARIS 12,75,112,0,4.0,Local industriel. commercial ou assimilé,709.0,0.0,258.0
